In [ ]:
#@title 1. Импорт библиотек и данных

from google.colab import files

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns; sns.set()

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler

In [ ]:
#uploaded = files.upload()

#!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#!kaggle competitions download -c urfu-engai-ml-2022-regression

#!unzip urfu-engai-ml-2022-regression.zip

In [ ]:
DF_train = pd.read_csv('/content/Train.csv', delimiter = ',', index_col = 'index') #данные для обучения
DF_test = pd.read_csv('/content/Test.csv', delimiter = ',', index_col = 'index') #данные для теста
Target = pd.read_csv('/content/Target.csv', delimiter = ',', index_col = 'index') #целевые значения
Submission = pd.read_csv('/content/Submission.csv', delimiter = ',', index_col = 'index') #форма для получившегося результата

DF_train

In [ ]:
#@title 2. Анализ данных

DF_combined = pd.concat([DF_train, DF_test]) #объединим тестовые и тренировочные данные для анализа и применения изменений
DF_combined.info()

In [ ]:
DF_combined = DF_combined.astype({'year': 'object'}) #Изменим тип данных для колонки year
DF_combined.dtypes

In [ ]:
cat_columns = [] #определим категориальные и числовые данные
num_columns = []

for column_name in DF_combined.columns:
    if (DF_combined[column_name].dtypes == object):
        cat_columns +=[column_name]
    else:
        num_columns +=[column_name]

print('Категориальные данные:\t ',cat_columns, '\n Число столблцов = ',len(cat_columns))
print('Числовые данные:\t ',  num_columns, '\n Число столблцов = ',len(num_columns))

In [ ]:
DF_combined.describe() #оценим числовые характеристики данных

In [ ]:
width = 2 #построим гистограммы данных
height = int(np.ceil(len(num_columns)/width))
fig, ax = plt.subplots(nrows=height, ncols=width, figsize=(20,40))

for idx, column_name in enumerate(num_columns):
    plt.subplot(height,width, idx+1)

    sns.histplot(data=DF_combined, x=column_name, bins = 5);

In [ ]:
colors_map = LinearSegmentedColormap.from_list(name = '', colors = ['red', 'white', 'green']) #оценим корреляцию между признаками и целевой переменной

DF_combined.join(Target).corr().style.background_gradient(cmap = colors_map, vmin = -1, vmax = 1)

Можно видеть, что признаки слабо коррелируют с целевой величиной, и так же нет однозначно дублирующих друг друга признаков, которые было бы целесообразно исключить.

##3. Проведем манипуляции с данными

Проведем нормализацию

In [ ]:
DF_combined_norm  = DF_combined.copy()
Xmin = DF_combined[num_columns].min()
Xmax = DF_combined[num_columns].max()

DF_combined_norm[num_columns] = (DF_combined[num_columns] - Xmin)/( Xmax - Xmin)

Пример данных до нормализации:

In [ ]:
fig = plt.figure(figsize=(10,7))
sns.histplot(data = DF_combined, x = 'population', bins = 5, kde = True, palette='bwr');

Те же данные после нормализации:

In [ ]:
fig = plt.figure(figsize=(10,7))
sns.histplot(data = DF_combined_norm, x = 'population', bins = 5, kde = True, palette='bwr');

Выполним one-hot encoding

In [ ]:
DF_combined = pd.get_dummies(DF_combined_norm, columns=cat_columns)
DF_combined.head()

##4. Обучение модели

In [ ]:
Train = DF_combined.iloc[0:DF_train.shape[0],:] #разделим снова данные на train и test
Test = DF_combined.iloc[DF_train.shape[0]:,:]

X_train, X_test, Y_train, Y_test = train_test_split(Train.values, Target.values.ravel(), test_size = 0.3, random_state = 42) #определим наборы данных для обучения и валидации

Обучим модель с регуляризацией Ridge

In [ ]:
alpha = 0.6

model_ridge = Ridge(alpha=alpha,max_iter=10000)
model_ridge.fit(X_train, Y_train)

print(f'Test Score = {model_ridge.score(X_test, Y_test):.2f}, Train Score = {model_ridge.score(X_train, Y_train):.2f}')

Test Score = 0.66, Train Score = 0.82


In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit

scoring = {'R2': 'r2',
           '-MSE': 'neg_mean_squared_error',
           '-MAE': 'neg_mean_absolute_error',
           'Max': 'max_error'}


scores = cross_validate(model_ridge, X_train, Y_train,
                      scoring=scoring, cv=ShuffleSplit(n_splits=5, random_state=42) )

print('Результаты Кросс-валидации')
DF_cv_linreg = pd.DataFrame(scores)
display(DF_cv_linreg)
print('\n')
print(DF_cv_linreg.mean()[2:])
print('\n')

Результаты Кросс-валидации


,fit_time,score_time,test_R2,test_-MSE,test_-MAE,test_Max
0,0.050491,0.001713,0.408027,-0.225722,-0.347874,-2.051612
1,0.077815,0.001686,0.561476,-0.145809,-0.298739,-1.005273
2,0.079458,0.001715,0.487501,-0.206172,-0.317270,-2.499225
3,0.074416,0.004326,0.624683,-0.173772,-0.326953,-1.317331
4,0.072977,0.004332,0.585481,-0.185634,-0.322612,-1.838283




test_R2      0.533434
test_-MSE   -0.187422
test_-MAE   -0.322689
test_Max    -1.742345
dtype: float64




Получим прогноз на изначальном тренировочном датасете для нахождения искомого результата

In [ ]:
Y_predict = model_ridge.predict(Test.values)

Y_predict

array([ 1.28069207,  0.72132912,  1.459737  ,  1.53256424,  1.3241679 ,
        0.61933511,  0.58422808,  1.14914674,  1.39984803,  0.90422398,
        0.54054047,  1.56635289,  1.27163354,  0.49313945,  1.80883329,
        1.59631566,  1.84288185,  0.91057097,  0.88513632,  0.68028282,
        1.14598932,  1.73010144,  1.5415943 ,  1.04267298,  0.86938367,
        0.61562423,  0.7987333 ,  1.88727473,  0.79292879,  1.55737999,
        1.12538852,  0.14380164,  0.37008672,  1.37559732,  0.98613264,
        0.67053951,  1.16203794,  0.68367801,  1.12121519,  0.85434411,
        1.07922318,  1.71789116,  0.59371381,  0.71753137,  0.23144832,
        0.93666036,  1.13738698,  1.02234095,  1.54078814,  1.07413051,
        1.29926293,  0.65889154,  1.28252685,  0.89518865,  1.49589335,
        1.41977696,  0.82984838,  0.55985912,  1.74616872,  0.71503424,
        1.44784223,  1.19952838,  1.27163354,  0.93452708,  1.01877688,
        1.22425376,  1.33415053,  1.5110214 ,  1.12575419,  0.76

In [ ]:
Submission['polution'] = Y_predict
Submission.to_csv('Submission.csv', index=True)